# Pachete necesare pentru folosirea acestui Notebook

Vom folosi [scipy](https://scipy.org/), [numpy](https://numpy.org/) și [matplotlib](https://matplotlib.org/).

In [1]:
from scipy import misc, ndimage
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.io import wavfile
from IPython.display import Audio

ModuleNotFoundError: No module named 'cv2'

# Imaginea cu care lucrăm

Vom folosi o imagine din setul de date oferit implicit de către scipy.

In [ ]:
X = misc.face(gray=True)
plt.imshow(X, cmap=plt.cm.gray)
plt.show()

# Transformata Fourier a unei imagini

Transformata Fourier Discretă se extinde ușor la mai multe dimensiuni. Pentru un semnal bidimensional precum o imagine DFT devine:

$$
Y_{m_1,m_2} = \sum_{n_1=0}^{N_1-1}\sum_{n_2=0}^{N_2-1}x_{n_1,n_2}e^{-j2\pi(m_1 n_1/N_1 + m_2 n_2/N_2)}
$$

* unde $n_1$ și $n_2$ sunt pozițile pixelilor pe orizontală, respectiv, pe verticală
* bin-urile rezultate corespund pozițiilor pixelilor
* spectrul este în continuare simetric
* proprietățile transformatei DFT 1D sunt respectate și în cazul celei 2D

În continuare vom folosi rutina generală `fft2` ce servește mai bine activității de învățare, deși pentru semnale reale ar trebui să folosim `rfft2` ce întoarce doar informația esențială (ex. omite simetriile). De asemenea vom analiza spectrul în scală logaritmică pentru a diferenția mai bine magnitudinile bin-urilor DTF.

In [ ]:
Y = np.fft.fft2(X)
freq_db = 20*np.log10(abs(Y))

plt.imshow(freq_db)
plt.colorbar()
plt.show()

Operațiile efectuate direct asupra imaginii se reflectă și în spectrul acesteia. Iată un exemplu a unei rotații de 45 de grade:

In [ ]:
rotate_angle = 45
X45 = ndimage.rotate(X, rotate_angle)
plt.imshow(X45, cmap=plt.cm.gray)
plt.show()

Y45 = np.fft.fft2(X45)
plt.imshow(20*np.log10(abs(Y45)))
plt.colorbar()
plt.show()

Momentan pe axe sunt afișate numărul bin-urilor. Pentru a obține frecvențele asociate folosiți `fftfreq`:

In [ ]:
freq_x = np.fft.fftfreq(X.shape[1])
freq_y = np.fft.fftfreq(X.shape[0])

plt.stem(freq_x, freq_db[:][0])
plt.show()

# Atenuarea frecvențelor înalte

Pentru a anula frecvențele de peste un anumit prag `freq_cutoff` putem pur și simplu anula intrările din spectru și aplica transformata Fourier inversă:

In [ ]:
freq_cutoff = 120

Y_cutoff = Y.copy()
Y_cutoff[freq_db > freq_cutoff] = 0
X_cutoff = np.fft.ifft2(Y_cutoff)
X_cutoff = np.real(X_cutoff)    # avoid rounding erros in the complex domain,
                                # in practice use irfft2
plt.imshow(X_cutoff, cmap=plt.cm.gray)
plt.show()

# Zgomot

Zgomotul alb perturbă în mod egal spectrul semnalului. Este astfel egal distribuit și regăsit în toate bin-urile DFT. [Zgomotul color](https://en.wikipedia.org/wiki/Colors_of_noise) se schimbă de-a lungul frecvențelor.

Putem adăuga zgomot în limita a `pixel_noise` pixeli imaginii folosind `random.randint`:

In [ ]:
pixel_noise = 200

noise = np.random.randint(-pixel_noise, high=pixel_noise+1, size=X.shape)
X_noisy = X + noise
plt.imshow(X, cmap=plt.cm.gray)
plt.title('Original')
plt.show()
plt.imshow(X_noisy, cmap=plt.cm.gray)
plt.title('Noisy')
plt.show()

# Sarcini

1. [8p] Produceți imaginile și spectrul pentru funcțiile de mai jos și dați o explicație scurtă pentru fiecare rezultat.
* $x_{n_1,n_2} = \sin(2\pi n_1 + 3\pi n_2)$
* $x_{n_1,n_2} = \sin(4\pi n_1) + \cos(6\pi n_2)$
* $Y_{0,5} = Y_{0,N-5} = 1\text{, altfel }Y_{m_1,m_2} = 0,\ \forall m_1, m_2$
* $Y_{5,0} = Y_{N-5,0} = 1\text{, altfel }Y_{m_1,m_2} = 0,\ \forall m_1, m_2$
* $Y_{5,5} = Y_{N-5,N-5} = 1\text{, altfel }Y_{m_1,m_2} = 0,\ \forall m_1, m_2$

*Atenție*: $x$ reprezintă informație în domeniul timpului, $Y$ în domeninul frecvenței.

2. [4p] Comprimați imaginea cu ratonul de mai sus prin atenuarea frecvențelor înalte până la un prag SNR autoimpus.

3. [4p] Eliminați zgomotul adăugat la imaginea cu ratonul produsă mai sus. Prezentați raportul SNR înainte și după.

4. [4p] Alegeți o secvență scurtă de timp (ex. 5-10 secunde) și eliminați un instrument la alegere din semnalul audio rezultat în urma rezolvării sarcinilor de la [laboratorul 5](https://cs.unibuc.ro/~pirofti/ps/ps-lab-5.pdf).

## Sarcina 1

In [ ]:
# Definesc o functie prin care sa generez toate imaginile.
# Parametrii: f - functia imaginii, size - dimensiunile imaginii, step - pasul de esantionare.


# timp -> frecventa
def make_image(f, size, step):

    # creez "pixelii" imaginii, la fel cum generam time in 1D
    N1, N2 = np.mgrid[0:size:step, 0:size:step]

    # esantionez functia pe pixelii de mai sus
    y = f(N1, N2)

    # aplic transformata Fourier
    Y = np.fft.fft2(y)

    # scap de simetrii
    Y = Y[:Y.shape[0] // 2, :Y.shape[1] // 2]

    # transform in scara logaritmica
    Y_scaled = 20 * np.log10(np.abs(Y))

    fig, axs = plt.subplots(1, 2)

    axs[0].set_title("Imagine")
    axs[0].imshow(y)

    axs[1].set_title("Spectrul imaginii")
    axs[1].imshow(Y_scaled)

    plt.show()


a) $x_{n_1,n_2} = \sin(2\pi n_1 + 3\pi n_2)$

In [ ]:
def x1(n1, n2):
    return np.sin(2 * np.pi * n1 + 3 * np.pi * n2)

make_image(x1, 1, 1/30)


Semnalul este compus dintr-o serie de dungi paralele identice.

b) $x_{n_1,n_2} = \sin(4\pi n_1) + \cos(6\pi n_2)$

In [ ]:
def x2(n1, n2):
    return np.sin(4 * np.pi * n1) + np.cos(6 * np.pi * n2)

make_image(x2, 2, 1/10)

Imaginea este compusa din componente perpendiculare.

In [ ]:
# frecventa -> timp
def make_image(f, size, step):

    N1, N2 = np.mgrid[0:size:step, 0:size:step]

    Y = f(N1, N2)

    # aplic transformarea fourier inversa
    y = np.fft.ifft2(Y).real

    Y = Y[:Y.shape[0] // 2, :Y.shape[1] // 2]

    fig, axs = plt.subplots(1, 2)

    axs[0].set_title("Spectrul imaginii")
    axs[0].imshow(Y)

    axs[1].set_title("Imaginea")
    axs[1].imshow(y)

c) $Y_{0,5} = Y_{0,N-5} = 1\text{, altfel }Y_{m_1,m_2} = 0,\ \forall m_1, m_2$

In [ ]:
n = 30

def y3(n1, n2):
    condition = (n1 == 0) & ((n2 == 5) | (n2 == n - 5))
    return np.where(condition, 1, 0)

make_image(y3, n, 1)

Imaginea este formata din linii verticale.

d) $Y_{5,0} = Y_{N-5,0} = 1\text{, altfel }Y_{m_1,m_2} = 0,\ \forall m_1, m_2$

In [ ]:
n = 30

def y4(n1, n2):
    condition = ((n1 == 5) | (n1 == n - 5)) & (n2 == 0)
    return np.where(condition, 1, 0)

make_image(y4, n, 1)

Imaginea este formata din linii orizontale.

e) $Y_{5,5} = Y_{N-5,N-5} = 1\text{, altfel }Y_{m_1,m_2} = 0,\ \forall m_1, m_2$

In [ ]:
n = 50

def y5(n1, n2):
    condition = ((n1 == 5) & (n2 == 5)) | ((n1 == (n - 5)) & (n2 == (n - 5)))
    return np.where(condition, 1, 0)

make_image(y5, n, 1)

Imaginea este formata din linii oblice, observarm ca frecventa pe orizontala si verticala este 5, deci liniile sunt la 45 grade.

## Sarcina 2

In [ ]:
fig, axs = plt.subplots(1, 2)

X = misc.face(gray=True)
axs[0].imshow(X, cmap=plt.cm.gray)
axs[0].set_title("Imaginea originala cu ratonul")

prag = 130

Y = np.fft.rfft2(X)
Y_scaled = 20 * np.log10(np.abs(Y))
Y[Y_scaled > prag] = 0
Y_filtered = np.fft.irfft2(Y)

def find_psnr(original, filtered):
    sd = (original - filtered) ** 2
    mse = np.mean(sd)

    if mse == 0:
        return 0

    max_pixel_intensity = np.max(original)
    ratio = (max_pixel_intensity ** 2) / mse
    return 10 * np.log10(ratio)

psnr = find_psnr(X, Y_filtered)
axs[1].set_title(f"Prag = {prag}\nPSNR = {psnr:.4f} dB")
axs[1].imshow(Y_filtered, cmap=plt.cm.gray)

plt.show()

## Sarcina 3

In [ ]:
pixel_noise = 200

noise = np.random.randint(-pixel_noise, high=pixel_noise+1, size=X.shape)
X_noisy = X + noise

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0][0].imshow(X, cmap=plt.cm.gray)
axs[0][0].set_title('Original')

axs[0][1].imshow(X_noisy, cmap=plt.cm.gray)
psnr = find_psnr(X, X_noisy)
axs[0][1].set_title(f"Noisy\nPSNR = {psnr:.4f} dB")

# blurez imagineaz pentru a scapa de zgomot
# folosesc ferestre de tip mean filter, una de 3x3 si una de 5x5 pentru a observa diferentele

X_blured3 = cv2.blur(X_noisy, (3, 3))
X_blured5 = cv2.blur(X_noisy, (5, 5))

axs[1][0].imshow(X_blured3, cmap=plt.cm.gray)
psnr = find_psnr(X, X_blured3)
axs[1][0].set_title(f"Blured\nPSNR = {psnr:.4f} dB")

axs[1][1].imshow(X_blured5, cmap=plt.cm.gray)
psnr = find_psnr(X, X_blured5)
axs[1][1].set_title(f"Blured\nPSNR = {psnr:.4f} dB")


plt.tight_layout()
plt.show()

## Sarcina 4

In [ ]:
# primele 5 secunde
with open("lab5_trimmed1.wav", 'rb') as audio:
    rate, data = wavfile.read(audio)

display(Audio(data.T, rate=rate))

Y = np.fft.rfft(data)

Y[:10000] = 0


X = np.fft.irfft(Y)

display(Audio(X.T, rate=rate))
# Nu se mai aude toba mare in audio
